## Homework 1 - Part 1 
## Implementation of the Travelling Salesman Problem using Google ORTools
####  Name : Sowmya Sivakumar
#### Andrew ID : sowmyasi@andrew.cmu.edu

#### Objective : To find the shortest cirtcuit route starting from a given city, while visiting each city and returning to the origin

In [1]:
import numpy as np
import math 
import os
from ortools.constraint_solver import pywrapcp 
from ortools.constraint_solver import routing_enums_pb2

In [2]:
#read locations into numpy array 
locations = np.loadtxt("p1-locations.txt", dtype = 'str', delimiter = '\n')

#read distance matrix into numpy matrix
distanceMatrix = np.genfromtxt("p1-distances.csv", dtype = int, delimiter = ",")


In [3]:
#creating the data model 
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = distanceMatrix
    data['num_vehicles'] = 1
    data['depot'] = 1
    return data

In [4]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

In [5]:
def main():

    #create data model     
    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)
    
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    
      # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)


In [6]:
  
if __name__ == '__main__':
    main()

Objective: 42980 miles
Route for vehicle 0:
 1 -> 4 -> 7 -> 5 -> 6 -> 2 -> 9 -> 0 -> 3 -> 8 -> 1





#### The shortest possible circuit route starting from Teide, Spain that visits each city and returns to the origin is as follows :
#### Teide, Spain -> Montreal, Canada -> Brisbane, Australia -> Itsukushima Shrine, Japan -> Shanghai, China ->
#### ->Tallinn, Estonia -> Stockholm, Sweden -> St. Stephan's Cathedral, Austria -> Brugge, Belgium -> Edinburgh, Scotland -> Teide, Spain
#### The route distance is 42,980 miles
####  It uses the PATH_CHEAPEST_ARC parameter, which is the search mechanism used in this case. Starting from a route "start" node, it connects to the node which produces the cheapest route segment, then extends the route by iterating on the last node added to the route. It is a lowest-cost-first-search mechanism.
